In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
import tensorboard

In [2]:
# Model 1: from research paper

In [3]:
batch_size = 32
img_height = 128
img_width = 128

In [4]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding= 'same', input_shape=(img_height, img_width, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=8, activation = 'softmax'))

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [4]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid_processed"

datagen = ImageDataGenerator(rescale = 1./255) # The imported data will br normalized here.

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as training data

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as testing data

validation_generator = datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as validation data

Found 72096 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.
Found 3796 images belonging to 8 classes.


In [7]:
r = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 50,
)

Epoch 1/50


C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_16352/3220806991.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


2253/2253 [==============================] - 153s 63ms/step - loss: 1.6292 - accuracy: 0.3880 - val_loss: 1.3437 - val_accuracy: 0.5379
Epoch 2/50
2253/2253 [==============================] - 138s 61ms/step - loss: 1.2803 - accuracy: 0.5253 - val_loss: 1.1568 - val_accuracy: 0.5662
Epoch 3/50
2253/2253 [==============================] - 144s 64ms/step - loss: 1.0120 - accuracy: 0.6162 - val_loss: 1.0282 - val_accuracy: 0.6022
Epoch 4/50
2253/2253 [==============================] - 120s 53ms/step - loss: 0.8880 - accuracy: 0.6563 - val_loss: 0.9577 - val_accuracy: 0.6380
Epoch 5/50
2253/2253 [==============================] - 126s 56ms/step - loss: 0.8174 - accuracy: 0.6828 - val_loss: 0.9834 - val_accuracy: 0.6245
Epoch 6/50
2253/2253 [==============================] - 115s 51ms/step - loss: 0.7783 - accuracy: 0.6969 - val_loss: 0.9961 - val_accuracy: 0.6131
Epoch 7/50
2253/2253 [==============================] - 109s 49ms/step - loss: 0.7402 - accuracy: 0.7105 - val_loss: 0.9529 - val

In [ ]:
#model.save('model_researchpaper_2019.h5')

In [5]:
#model = tf.keras.models.load_model('model_researchpaper_2019.h5')

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_14560/2098712288.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_generator.classes, y_pred))

0.05268703898840885


In [ ]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_generator.classes, y_pred))